In [ ]:
import pandas as pd
import numpy as np
import sim_plotter_2 as sp
from datetime import datetime, timedelta

# Load the data

file = ["2024-03-24", "2024-03-25", "2024-03-26"]

for csv_date in file:
    data = pd.read_csv(f"{csv_date}_packages.csv")

    filtered_df = data[data['sortation_center'] == 3858]
#filtered_df = filtered_df[filtered_df['scdre.scac'].str.contains('TLMD', na = False)]

    time_columns = ['pkg_sort_code_assigned_utc_ts',
                    'pkg_label_created_utc_ts',
                    'pkg_received_utc_ts',
                    'pkg_inducted_utc_ts',
                    'pkg_sorted_utc_ts',
                    'pkg_staged_utc_ts',
                    'pkg_loaded_utc_ts',
                    'pkg_outbound_utc_ts',
                    'pkg_critical_pull_time_utc_ts']
    for col in time_columns:
        if col in filtered_df.columns:
            filtered_df[col]=pd.to_datetime(filtered_df[col], errors = 'coerce')

    def convert_to_datetime(minutes, shift_start, date_sim):
        shift_start_dt = datetime.strptime(shift_start, "%H:%M")
        shift_start_minutes = shift_start_dt.hour * 60 + shift_start_dt.minute
        total_minutes = shift_start_minutes + minutes
        
        if total_minutes >= 24 * 60:
            total_minutes -= 24 * 60
        
        if total_minutes < shift_start_minutes:
            date = date_sim + timedelta(days=1)
        else:
            date = date_sim
        
        result_datetime = datetime.combine(date.date(), datetime.min.time()) + timedelta(minutes=total_minutes)
        return result_datetime
        
    filtered_df = filtered_df[filtered_df['scdre.scac'].str.contains('TLMD', na = False)]
    filtered_df = filtered_df.drop(columns = ['sortation_center'])
    filtered_df = filtered_df.drop(columns = ['scdre.ship_d'])
    filtered_df = filtered_df.drop(columns = ['scdre.scac'])
    filtered_df = filtered_df.drop(columns = ['pkg_sort_code_assigned_utc_ts'])
    filtered_df = filtered_df.drop(columns = ['pkg_label_created_utc_ts'])
    filtered_df = filtered_df.drop(columns = ['scdre.location_id'])
    filtered_df = filtered_df.drop(columns = ['scdre.route_sort_c'])
    filtered_df = filtered_df.drop(columns = ['scdre.sortation_center_package_sort_c'])
    filtered_df = filtered_df.drop(columns = ['scdre.sortation_center_package_inbound_pallet_id'])
    filtered_df = filtered_df.drop(columns = ['scdre.sortation_center_package_sort_partition'])
    filtered_df = filtered_df.drop(columns = ['pkg_cancelled_utc_ts'])
    filtered_df = filtered_df.drop(columns = ['pkg_outbound_utc_ts'])
    filtered_df = filtered_df.drop(columns = ['pkg_critical_pull_time_utc_ts'])


    df_1 = filtered_df
    df_2 = filtered_df[filtered_df['scdre.scac'].str.contains('TLMD', na = False)]
    date_simulated = datetime.strptime(csv_date, "%m/%d/%Y")




    shift_start = "16:30"
    df_1['received_ts'] = df_1['pkg_received_utc_ts'].apply(lambda x: convert_to_datetime(x, shift_start,csv_date))
    df_1['inducted_ts'] = df_1['pkg_inducted_utc_ts'].apply(lambda x: convert_to_datetime(x, shift_start,csv_date))
    df_1['sorted_ts'] = df_1['pkg_sorted_utc_ts'].apply(lambda x: convert_to_datetime(x, shift_start,csv_date))
    df_2['sorted_ts'] = df_2['pkg_sorted_utc_ts'].apply(lambda x: convert_to_datetime(x, shift_start,csv_date))
    

    df_long = df_1.melt(var_name='timestamp_type', value_name='timestamp', ignore_index=False)
    df_long.set_index('timestamp', inplace=True)

    df_long['count'] = 1
    pivot_table = df_long.pivot_table(index='timestamp', columns='timestamp_type', values='count', aggfunc='sum', fill_value=0)

    pivot_table = pivot_table.reset_index()

    pivot_table['timestamp'] = pd.to_datetime(pivot_table['timestamp'], errors='coerce')

    pivot_table['received_ts_cum'] = pivot_table['received_ts'].cumsum() - pivot_table['inducted_ts'].cumsum()
    pivot_table['Received'] = pivot_table['received_ts_cum']

    # Combine `timestamp` and `Received` into a new DataFrame
    combined_df = pd.DataFrame({
        'timestamp': pivot_table['timestamp'],
        'Received': pivot_table['Received']
    })

    # Set 'timestamp' as the index


    combined_df.set_index('timestamp', inplace=True)
    combined_df.to_csv(f'{csv_date}_received_ts.csv')


    #induct
    df_long = df_1.melt(var_name='timestamp_type', value_name='timestamp', ignore_index=False)
    df_long.set_index('timestamp', inplace=True)

    df_long['count'] = 1
    pivot_table = df_long.pivot_table(index='timestamp', columns='timestamp_type', values='count', aggfunc='sum', fill_value=0)

    pivot_table = pivot_table.reset_index()

    pivot_table['timestamp'] = pd.to_datetime(pivot_table['timestamp'], errors='coerce')

    pivot_table['inducted_ts_cum'] = pivot_table['inducted_ts'].cumsum() - pivot_table['sorted_ts'].cumsum()
    pivot_table['Inducted'] = pivot_table['inducted_ts_cum']

    # Combine `timestamp` and `Received` into a new DataFrame
    combined_df_2 = pd.DataFrame({
        'timestamp': pivot_table['timestamp'],
        'Inducted': pivot_table['Inducted']
    })

    # Set 'timestamp' as the index


    combined_df_2.set_index('timestamp', inplace=True)
    combined_df_2.to_csv(f'{csv_date}_inducted_ts.csv')

    #sort
    df_long = df_2.melt(var_name='timestamp_type', value_name='timestamp', ignore_index=False)
    df_long.set_index('timestamp', inplace=True)

    df_long['count'] = 1
    pivot_table = df_long.pivot_table(index='timestamp', columns='timestamp_type', values='count', aggfunc='sum', fill_value=0)

    pivot_table = pivot_table.reset_index()

    pivot_table['timestamp'] = pd.to_datetime(pivot_table['timestamp'], errors='coerce')


    pivot_table['sorted_ts_cum'] = pivot_table['sorted_ts'].cumsum()
    pivot_table['Sorted'] = pivot_table['sorted_ts_cum']

    # Combine `timestamp` and `Received` into a new DataFrame
    combined_df_3 = pd.DataFrame({
        'timestamp': pivot_table['timestamp'],
        'Sorted': pivot_table['Sorted']
    })

    # Set 'timestamp' as the index


    combined_df_3.set_index('timestamp', inplace=True)
    combined_df_3.to_csv(f'{csv_date}_sorted_ts.csv')


C:\Users\fenst\AppData\Local\Temp\ipykernel_14172\505581398.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col]=pd.to_datetime(filtered_df[col], errors = 'coerce')
C:\Users\fenst\AppData\Local\Temp\ipykernel_14172\505581398.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[col]=pd.to_datetime(filtered_df[col], errors = 'coerce')
C:\Users\fenst\AppData\Local\Temp\ipykernel_14172\505581398.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

KeyError: 'scac'